In [ ]:
import requests
import pandas as pd
import time

api_key = "57adf7257cecfd7731d11a3a7deb0ae3"
movie_list = []

for year in range(2021, 2025):
    print(f"===== {year}년 영화 데이터 수집 중 =====")
    cur_page = 1
    total_page = 1

    while cur_page <= total_page:
        url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
        params = {
            "key": api_key,
            "openStartDt": year,
            "itemPerPage": 100,
            "curPage": cur_page
        }
        try:
            res = requests.get(url, params=params, timeout=30)
            res.raise_for_status()
            data = res.json()
        except Exception as e:
            print(f"오류: {e} (페이지 {cur_page})")
            time.sleep(2)
            cur_page += 1
            continue

        movies = data['movieListResult']['movieList']
        total_cnt = int(data['movieListResult']['totCnt'])
        total_page = (total_cnt - 1) // 100 + 1

        for m in movies:
            nation = m.get('nationAlt', '').lower()
            openDt = m.get('openDt', '')
            # 날짜 파싱 및 필터링
            if openDt and (len(openDt) == 8 or len(openDt) == 10):
                # YYYYMMDD → YYYY-MM-DD로 변환
                if len(openDt) == 8:
                    date_str = f"{openDt[:4]}-{openDt[4:6]}-{openDt[6:]}"
                else:
                    date_str = openDt
                try:
                    y, mth, d = map(int, date_str.split('-'))
                    # 2021-01-01 ~ 2025-07-31 범위만 저장
                    if (
                        (y == 2021 and mth >= 1) or
                        (y in [2022, 2023, 2024]) or
                        (y == 2025 and mth <= 7)
                    ):
                        if '한국' in nation or 'korea' in nation:
                            movie_list.append({
                                "movieCd": m.get('movieCd', ''),
                                "movieNm": m.get('movieNm', ''),
                                "openDt": date_str,
                                "repGenreNm": m.get('repGenreNm', ''),
                                "directors": ", ".join([d['peopleNm'] for d in m.get('directors', [])]),
                                "prdtYear": m.get('prdtYear', ''),
                                "nationAlt": m.get('nationAlt', ''),
                                "watchGradeNm": m.get('watchGradeNm', '')
                            })
                except Exception as e:
                    continue  # 날짜 형식 이상치 건너뜀

        print(f"{year}년 {cur_page}/{total_page}페이지 (누적 {len(movie_list)}편)")
        cur_page += 1
        time.sleep(1)

# 최종 결과 저장
df = pd.DataFrame(movie_list)
df.to_csv('kobis_KOREAN_movies_202101_202507.csv', index=False, encoding='utf-8-sig')
print('✅ 2021년 ~ 2025년 7월 한국영화만 저장 완료! (kobis_KOREAN_movies_202101_202507.csv)')


===== 2021년 영화 데이터 수집 중 =====
2021년 1/74페이지 (누적 28편)
2021년 2/74페이지 (누적 77편)
2021년 3/74페이지 (누적 116편)
2021년 4/74페이지 (누적 163편)
2021년 5/74페이지 (누적 206편)
2021년 6/74페이지 (누적 248편)
2021년 7/74페이지 (누적 295편)
2021년 8/74페이지 (누적 350편)
2021년 9/74페이지 (누적 395편)
2021년 10/74페이지 (누적 443편)
2021년 11/74페이지 (누적 498편)
2021년 12/74페이지 (누적 545편)
2021년 13/74페이지 (누적 583편)
2021년 14/74페이지 (누적 629편)
2021년 15/74페이지 (누적 668편)
2021년 16/74페이지 (누적 718편)
2021년 17/74페이지 (누적 776편)
2021년 18/74페이지 (누적 819편)
2021년 19/74페이지 (누적 868편)
2021년 20/74페이지 (누적 906편)
2021년 21/74페이지 (누적 962편)
2021년 22/74페이지 (누적 1002편)
2021년 23/74페이지 (누적 1042편)
2021년 24/74페이지 (누적 1087편)
2021년 25/74페이지 (누적 1146편)
2021년 26/74페이지 (누적 1197편)
2021년 27/74페이지 (누적 1232편)
2021년 28/74페이지 (누적 1261편)
2021년 29/74페이지 (누적 1298편)
2021년 30/74페이지 (누적 1344편)
2021년 31/74페이지 (누적 1381편)
2021년 32/74페이지 (누적 1432편)
2021년 33/74페이지 (누적 1478편)
2021년 34/74페이지 (누적 1537편)
2021년 35/74페이지 (누적 1579편)
2021년 36/74페이지 (누적 1609편)
2021년 37/74페이지 (누적 1644편)
2021년 38/74페이지 (누적 1679편)
2021년 39/74페이지

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

api_key = "57adf7257cecfd7731d11a3a7deb0ae3"

# 1. 박스오피스 누적 정보 저장용 딕셔너리
boxoffice_latest = {}

# 2. 날짜 범위 지정
start_date = datetime(2021, 1, 1)
end_date = datetime(2025, 7, 31)
cur_date = start_date

while cur_date <= end_date:
    date_str = cur_date.strftime('%Y%m%d')
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json"
    params = {
        "key": api_key,
        "targetDt": date_str,
        "itemPerPage": 10  # 일일 박스오피스는 기본 10위
    }
    try:
        res = requests.get(url, params=params, timeout=10)
        res.raise_for_status()
        data = res.json()
        daily_list = data.get('boxOfficeResult', {}).get('dailyBoxOfficeList', [])
        for movie in daily_list:
            movieCd = movie.get('movieCd')
            # 날짜가 뒤로 갈수록 최신값으로 덮어쓰기!
            boxoffice_latest[movieCd] = {
                'movieCd': movieCd,
                'movieNm': movie.get('movieNm', ''),
                'audiAcc': movie.get('audiAcc', ''),      # 누적 관객수
                'salesAcc': movie.get('salesAcc', ''),    # 누적 매출액
                'targetDt': date_str
            }
    except Exception as e:
        print(f"{date_str} 에러: {e}")

    # 진행상황 출력, 딜레이
    if cur_date.day == 1:
        print(f"{date_str} 처리 중 (누적 {len(boxoffice_latest)}편)")
    time.sleep(0.5)
    cur_date += timedelta(days=1)

# 3. 데이터프레임으로 변환
boxoffice_df = pd.DataFrame(list(boxoffice_latest.values()))
boxoffice_df.to_csv('kobis_boxoffice_latest_202101_202507.csv', index=False, encoding='utf-8-sig')
print("✅ 박스오피스 수집 완료! (순위권 영화만 포함)")

# 4. (옵션) 기존 한국영화 리스트와 movieCd 기준 병합
# 기존 한국영화 csv: kobis_KOREAN_movies_202101_202507.csv
korean_df = pd.read_csv('kobis_KOREAN_movies_202101_202507.csv', encoding='utf-8-sig')
merged_df = korean_df.merge(boxoffice_df[['movieCd', 'audiAcc', 'salesAcc', 'targetDt']], on='movieCd', how='left')

merged_df.to_csv('kobis_KOREAN_movies_202101_202507_with_boxoffice.csv', index=False, encoding='utf-8-sig')
print("✅ 한국영화 전체 + 박스오피스 누적관객/매출 합치기 완료!")


20210101 처리 중 (누적 10편)
20210201 처리 중 (누적 51편)
20210301 처리 중 (누적 78편)
20210401 처리 중 (누적 110편)
20210501 처리 중 (누적 135편)
20210601 처리 중 (누적 160편)
20210701 처리 중 (누적 183편)
20210801 처리 중 (누적 205편)
20210901 처리 중 (누적 228편)
20211001 처리 중 (누적 246편)
20211101 처리 중 (누적 275편)
20211201 처리 중 (누적 300편)
20220101 처리 중 (누적 324편)
20220201 처리 중 (누적 348편)
20220301 처리 중 (누적 365편)
20220401 처리 중 (누적 390편)
20220501 처리 중 (누적 414편)
20220601 처리 중 (누적 444편)
20220701 처리 중 (누적 462편)
20220801 처리 중 (누적 483편)
20220901 처리 중 (누적 497편)
20221001 처리 중 (누적 518편)
20221101 처리 중 (누적 535편)
20221201 처리 중 (누적 562편)
20230101 처리 중 (누적 577편)
20230201 처리 중 (누적 590편)
20230301 처리 중 (누적 608편)
20230401 처리 중 (누적 623편)
20230501 처리 중 (누적 645편)
20230601 처리 중 (누적 664편)
20230701 처리 중 (누적 688편)
20230801 처리 중 (누적 709편)
20230901 처리 중 (누적 727편)
20231001 처리 중 (누적 750편)
20231101 처리 중 (누적 779편)
20231201 처리 중 (누적 806편)
20240101 처리 중 (누적 827편)
20240201 처리 중 (누적 842편)
20240301 처리 중 (누적 864편)
20240401 처리 중 (누적 888편)
20240501 처리 중 (누적 908편)
20240601 처리 중 (누적 9

ValueError: You are trying to merge on int64 and object columns for key 'movieCd'. If you wish to proceed you should use pd.concat

In [3]:
import pandas as pd

# 1. 이미 저장된 CSV 파일 읽기
korean_df = pd.read_csv('kobis_KOREAN_movies_202101_202507.csv', encoding='utf-8-sig')
boxoffice_df = pd.read_csv('kobis_boxoffice_latest_202101_202507.csv', encoding='utf-8-sig')

# 2. movieCd 컬럼 타입 맞추기 (이게 핵심!)
korean_df['movieCd'] = korean_df['movieCd'].astype(str)
boxoffice_df['movieCd'] = boxoffice_df['movieCd'].astype(str)

# 3. 병합 (박스오피스 정보가 없는 영화는 NaN이 됨)
merged_df = korean_df.merge(
    boxoffice_df[['movieCd', 'audiAcc', 'salesAcc', 'targetDt']],
    on='movieCd', how='left'
)

# 4. 저장 (원하는 이름으로)
merged_df.to_csv('kobis_KOREAN_movies_202101_202507_with_boxoffice.csv', index=False, encoding='utf-8-sig')
print("✅ 한국영화 전체 + 박스오피스 누적관객/매출 합치기 완료!")


✅ 한국영화 전체 + 박스오피스 누적관객/매출 합치기 완료!
